In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from scipy import stats
import torch

from common import load_x264, split_data, split_data_cv, evaluate_ii, evaluate_cc, prepare_result_df

In [2]:
## Configuration
random_seed = 33154

# Enter names of performance columns to consider
performances = ["rel_kbs"]

# Number of nearest neighbours to consider
# Make multiples to allow better budget comparison
topk_values = (1, 3, 5, 15, 25)
topr_values = (1, 3, 5, 15, 25)

data_dir = "../data"

In [3]:
## Load and prepare data
perf_matrix, input_features, config_features, all_performances = load_x264(
    data_dir=data_dir
)

print(f"Loaded data x264")
print(f"perf_matrix:{perf_matrix.shape}")
print(f"input_features:{input_features.shape}")
print(f"config_features:{config_features.shape}")

data_split = split_data(perf_matrix, random_state=random_seed)
train_inp = data_split["train_inp"]
train_cfg = data_split["train_cfg"]
test_inp = data_split["test_inp"]
test_cfg = data_split["test_cfg"]

# This is a look up for performance measurements from inputname + configurationID
input_config_map = (
    perf_matrix[["inputname", "configurationID"] + performances]
    .sort_values(["inputname", "configurationID"])
    .set_index(["inputname", "configurationID"])
)
all_input_names = pd.Series(
    input_config_map.index.get_level_values("inputname").unique()
)
all_config_ids = pd.Series(
    input_config_map.index.get_level_values("configurationID").unique()
)

regret_map = input_config_map.groupby("inputname").transform(
    lambda x: (x - x.min()).abs() / abs(x.min())
)
average_mape = regret_map.mean(axis=1)

rank_map = input_config_map.groupby("inputname").transform(
    lambda x: stats.rankdata(x, method="min")
)
average_ranks = rank_map.mean(axis=1)

Loaded data x264
perf_matrix:(258687, 45)
input_features:(1287, 21)
config_features:(201, 48)
Training data: 63.64%
Both new: 4.09%
Config new: 16.31%
Input new: 15.96%


In [4]:
## Prepare necessary torch tensors
# Prepare lookup tables for input/configuration performances as torch tensors
rank_arr = torch.from_numpy(
    rank_map.reset_index()  # .loc[(train_inp, train_cfg), :]
    .pivot_table(index="inputname", columns="configurationID", values=performances[0])
    .values
)
regret_arr = torch.from_numpy(
    regret_map.reset_index()  # .loc[(train_inp, train_cfg), :]
    .pivot_table(index="inputname", columns="configurationID", values=performances[0])
    .values
)

# Prepare and select training/test data according to random split
input_arr = torch.from_numpy(input_features.values).float()
config_arr = torch.from_numpy(config_features.values).float()

train_input_mask = input_features.index.isin(train_inp)
test_input_mask = input_features.index.isin(test_inp)

train_config_mask = config_features.index.isin(train_cfg)
test_config_mask = config_features.index.isin(test_cfg)

train_input_arr = input_arr[train_input_mask]
train_config_arr = config_arr[train_config_mask]

In [5]:
train_cc_rank = []
train_cc_ratio = []
train_cc_regret = []

test_cc_rank = []
test_cc_ratio = []
test_cc_regret = []

train_ii_rank = []
train_ii_ratio = []
train_ii_regret = []

test_ii_rank = []
test_ii_ratio = []
test_ii_regret = []

# Query: test data
# Database: train data

for topk in topk_values:
    train_cc = evaluate_cc(
            config_arr,
            rank_arr=rank_arr,
            regret_arr=regret_arr,
            n_neighbors=topk,
            n_recs=topr_values,
        query_mask=torch.from_numpy(train_config_mask),
        reference_mask=torch.from_numpy(train_config_mask)
    )
    train_cc_rank.append(train_cc[0].numpy())
    train_cc_regret.append(train_cc[1].numpy())
    train_cc_ratio.append(train_cc[2].numpy())

    test_cc = evaluate_cc(
            config_arr,
            rank_arr=rank_arr,
            regret_arr=regret_arr,
            n_neighbors=topk,
            n_recs=topr_values,
        query_mask=torch.from_numpy(test_config_mask),
        reference_mask=torch.from_numpy(train_config_mask)
    )    
    test_cc_rank.append(test_cc[0].numpy())
    test_cc_regret.append(test_cc[1].numpy())
    test_cc_ratio.append(test_cc[2].numpy())

    train_ii = evaluate_ii(
        input_arr,
        rank_arr=rank_arr,
        regret_arr=regret_arr,
        n_neighbors=topk,
        n_recs=topr_values,  
        query_mask=torch.from_numpy(train_input_mask),
        reference_mask=torch.from_numpy(train_input_mask)
    )
    train_ii_rank.append(train_ii[0].numpy())
    train_ii_regret.append(train_ii[1].numpy())
    train_ii_ratio.append(train_ii[2].numpy())

    test_ii = evaluate_ii(
        input_arr,
        rank_arr=rank_arr,
        regret_arr=regret_arr,
        n_neighbors=topk,
        n_recs=topr_values,
        query_mask=torch.from_numpy(test_input_mask),
        reference_mask=torch.from_numpy(train_input_mask)
    )
    test_ii_rank.append(test_ii[0].numpy())
    test_ii_regret.append(test_ii[1].numpy())
    test_ii_ratio.append(test_ii[2].numpy())

In [6]:


# TODO Share results in README

print("train cc ratio\n", prepare_result_df(train_cc_ratio, topr_values, topk_values), "\n")
print("train cc best rank\n", prepare_result_df(train_cc_rank, topr_values, topk_values), "\n")
print("train cc best regret\n", prepare_result_df(train_cc_regret, topr_values, topk_values), "\n")

print("test cc ratio\n", prepare_result_df(test_cc_ratio, topr_values, topk_values), "\n")
print("test cc best rank\n", prepare_result_df(test_cc_rank, topr_values, topk_values), "\n")
print("test cc best regret\n", prepare_result_df(test_cc_regret, topr_values, topk_values), "\n")

print("train ii ratio\n", prepare_result_df(train_ii_ratio, topr_values, topk_values), "\n")
print("train ii best rank\n", prepare_result_df(train_ii_rank, topr_values, topk_values), "\n")
print("train ii best regret\n", prepare_result_df(train_ii_regret, topr_values, topk_values), "\n")

print("test ii ratio\n", prepare_result_df(test_ii_ratio, topr_values, topk_values), "\n")
print("test ii best rank\n", prepare_result_df(test_ii_rank, topr_values, topk_values), "\n")
print("test ii best regret\n", prepare_result_df(test_ii_regret, topr_values, topk_values), "\n")

train cc ratio
             r                                            
            1          3          5         15         25
k                                                        
1         NaN        NaN        NaN        NaN        NaN
3   22.187502  29.270834  29.624998  35.020836  37.300003
5   24.687498  30.781252  31.843752  38.635414  42.150002
15  27.991074  35.223217  38.312500  45.258926  49.912498
25  29.895836  37.690971  41.187500  49.347221  54.714584 

train cc best rank
            r                                        
           1         3         5        15        25
k                                                   
1   7.064880  7.149217  7.222903  7.303484  7.411753
3   7.202473  7.225513  7.300764  7.415782  7.508535
5   7.105088  7.115968  7.175777  7.320344  7.419367
15  6.944768  7.060562  7.118841  7.274037  7.373463
25  7.174669  7.264842  7.308834  7.413736  7.501832 

train cc best regret
             r                                     

In [7]:
# Run cross-validation over all train/test splits
dfs = []

for data_split in split_data_cv(perf_matrix, random_state=random_seed):
    train_inp = data_split["train_inp"]
    train_cfg = data_split["train_cfg"]
    test_inp = data_split["test_inp"]
    test_cfg = data_split["test_cfg"]

    # Prepare and select training/test data according to random split
    input_arr = torch.from_numpy(input_features.values).float()
    config_arr = torch.from_numpy(config_features.values).float()

    train_input_mask = input_features.index.isin(train_inp)
    test_input_mask = input_features.index.isin(test_inp)

    train_config_mask = config_features.index.isin(train_cfg)
    test_config_mask = config_features.index.isin(test_cfg)

    train_input_arr = input_arr[train_input_mask]
    train_config_arr = config_arr[train_config_mask]

    train_cc_rank = []
    train_cc_ratio = []
    train_cc_regret = []

    test_cc_rank = []
    test_cc_ratio = []
    test_cc_regret = []

    train_ii_rank = []
    train_ii_ratio = []
    train_ii_regret = []

    test_ii_rank = []
    test_ii_ratio = []
    test_ii_regret = []

    # Query: test data
    # Database: train data

    for topk in topk_values:
        train_cc = evaluate_cc(
                config_arr,
                rank_arr=rank_arr,
                regret_arr=regret_arr,
                n_neighbors=topk,
                n_recs=topr_values,
            query_mask=torch.from_numpy(train_config_mask),
            reference_mask=torch.from_numpy(train_config_mask)
        )
        train_cc_rank.append(train_cc[0].numpy())
        train_cc_regret.append(train_cc[1].numpy())
        train_cc_ratio.append(train_cc[2].numpy())

        test_cc = evaluate_cc(
                config_arr,
                rank_arr=rank_arr,
                regret_arr=regret_arr,
                n_neighbors=topk,
                n_recs=topr_values,
            query_mask=torch.from_numpy(test_config_mask),
            reference_mask=torch.from_numpy(train_config_mask)
        )    
        test_cc_rank.append(test_cc[0].numpy())
        test_cc_regret.append(test_cc[1].numpy())
        test_cc_ratio.append(test_cc[2].numpy())

        train_ii = evaluate_ii(
            input_arr,
            rank_arr=rank_arr,
            regret_arr=regret_arr,
            n_neighbors=topk,
            n_recs=topr_values,  
            query_mask=torch.from_numpy(train_input_mask),
            reference_mask=torch.from_numpy(train_input_mask)
        )
        train_ii_rank.append(train_ii[0].numpy())
        train_ii_regret.append(train_ii[1].numpy())
        train_ii_ratio.append(train_ii[2].numpy())

        test_ii = evaluate_ii(
            input_arr,
            rank_arr=rank_arr,
            regret_arr=regret_arr,
            n_neighbors=topk,
            n_recs=topr_values,
            query_mask=torch.from_numpy(test_input_mask),
            reference_mask=torch.from_numpy(train_input_mask)
        )
        test_ii_rank.append(test_ii[0].numpy())
        test_ii_regret.append(test_ii[1].numpy())
        test_ii_ratio.append(test_ii[2].numpy())

    dfs.append(prepare_result_df(train_cc_rank, topr_values, topk_values, {"metric": "rank", "mode": "cc", "split": "train"}))
    dfs.append(prepare_result_df(train_cc_regret, topr_values, topk_values, {"metric": "regret", "mode": "cc", "split": "train"}))
    dfs.append(prepare_result_df(train_cc_ratio, topr_values, topk_values, {"metric": "ratio", "mode": "cc", "split": "train"}))

    dfs.append(prepare_result_df(test_cc_rank, topr_values, topk_values, {"metric": "rank", "mode": "cc", "split": "test"}))
    dfs.append(prepare_result_df(test_cc_regret, topr_values, topk_values, {"metric": "regret", "mode": "cc", "split": "test"}))
    dfs.append(prepare_result_df(test_cc_ratio, topr_values, topk_values, {"metric": "ratio", "mode": "cc", "split": "test"}))

    dfs.append(prepare_result_df(train_ii_rank, topr_values, topk_values, {"metric": "rank", "mode": "ii", "split": "train"}))
    dfs.append(prepare_result_df(train_ii_regret, topr_values, topk_values, {"metric": "regret", "mode": "ii", "split": "train"}))
    dfs.append(prepare_result_df(train_ii_ratio, topr_values, topk_values, {"metric": "ratio", "mode": "ii", "split": "train"}))

    dfs.append(prepare_result_df(test_ii_rank, topr_values, topk_values, {"metric": "rank", "mode": "ii", "split": "test"}))
    dfs.append(prepare_result_df(test_ii_regret, topr_values, topk_values, {"metric": "regret", "mode": "ii", "split": "test"}))
    dfs.append(prepare_result_df(test_ii_ratio, topr_values, topk_values, {"metric": "ratio", "mode": "ii", "split": "test"}))

full_df = pd.concat(dfs)
full_df.groupby(["mode", "split", "metric", "k"]).mean()

Training data: 55.96%
Both new: 6.35%
Config new: 19.02%
Input new: 18.67%
Training data: 56.33%
Both new: 6.22%
Config new: 18.65%
Input new: 18.80%
Training data: 56.33%
Both new: 6.22%
Config new: 18.65%
Input new: 18.80%
Training data: 56.39%
Both new: 6.20%
Config new: 18.67%
Input new: 18.74%


r                                      \
                               1           3           5          15   
mode split metric k                                                    
cc   test  rank   1    10.626880   10.690459   10.687014   10.635697   
                  3    10.598349   10.656416   10.652812   10.666245   
                  5    10.625179   10.686196   10.689018   10.679218   
                  15   10.775593   10.842163   10.853754   10.789198   
                  25   10.705626   10.775327   10.777406   10.748225   
           ratio  1          NaN         NaN         NaN         NaN   
                  3    17.416668   24.839869   26.117647   33.022549   
                  5    17.666668   25.030228   26.646568   34.155884   
                  15   24.836136   33.470825   36.368908   43.970123   
                  25   26.379492   36.194855   39.445015   48.364132   
           regret 1   149.076706  159.262115  176.631775  162.894867   
                  3   156.940231  161.910233  165.093750  157.735352   
                  5   154.878036  162.451294  165.446747  156.651016   
                  15  151.333099  163.553604  171.643097  159.744583   
                  25  143.170364  154.586411  159.003082  150.390396   
     train rank   1     7.293846    7.386559    7.504650    7.554715   
                  3     7.455570    7.558251    7.651608    7.676747   
                  5     7.371895    7.498907    7.580076    7.629185   
                  15    7.124954    7.251328    7.322071    7.415790   
                  25    7.284796    7.382918    7.446532    7.546454   
           ratio  1          NaN         NaN         NaN         NaN   
                  3    17.507175   24.551510   25.696026   32.211994   
                  5    19.538082   26.623344   28.132891   35.199375   
                  15   24.693237   33.293175   36.029518   43.543968   
                  25   25.261818   35.584885   38.549431   47.587250   
           regret 1    99.525162  101.973541  115.543800  107.161102   
                  3   111.364479  111.823502  116.891403  107.571754   
                  5   109.387756  111.543564  117.487228  107.788170   
                  15   99.025543  103.806412  112.788429  104.278183   
                  25   92.668488   99.650482  106.369492   98.729355   
ii   test  rank   1    26.095520   19.139107   16.214344   10.956704   
                  3    10.991971    7.434736    6.399575    3.603939   
                  5     6.870597    4.437309    3.657968    1.797710   
                  15    2.266540    1.167227    0.885457    0.389988   
                  25    1.225682    0.610293    0.434026    0.158504   
           ratio  1          NaN         NaN         NaN         NaN   
                  3    23.777355   29.849945   33.146732   44.232487   
                  5    31.856909   38.460842   42.314850   53.906521   
                  15   50.899612   57.815842   61.976864   73.353027   
                  25   60.123775   66.176559   70.051811   80.574661   
           regret 1    43.953884   34.372993   29.746437   21.238874   
                  3    22.111717   16.277683   14.604990    8.690106   
                  5    15.779476   11.246897    9.917996    5.287583   
                  15    6.322211    3.660258    3.171066    1.664403   
                  25    3.904664    2.002583    1.549987    0.662804   
     train rank   1    25.273321   18.832581   16.009808   10.532163   
                  3    10.179598    6.843678    5.759306    3.324680   
                  5     6.275717    3.948329    3.192158    1.744146   
                  15    1.901177    1.067198    0.819643    0.418388   
                  25    1.101451    0.531653    0.403955    0.171495   
           ratio  1          NaN         NaN         NaN         NaN   
                  3    24.669756   30.609753   33.615776   43.996666   
                  5    32.232544   38.716431   42.212040   53.877357   

In [43]:
# full_df.groupby(["mode", "split", "metric", "k"]).mean().to_clipboard()
dfmean = full_df.reset_index().groupby(["mode", "split", "metric", "k"], as_index=False).mean()
dfmean.to_csv("knn_config_recommendation.csv")

/tmp/ipykernel_916220/2456008311.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dfmean = full_df.reset_index().groupby(["mode", "split", "metric", "k"], as_index=False).mean()


In [44]:
latexstr = dfmean.query("mode == 'cc' & split == 'test' & metric == 'rank'") #.drop(columns=["mode", "split", "metric"]).to_latex(index=False)
print(latexstr)

UndefinedVariableError: name 'mode' is not defined

In [60]:
print(dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", caption="Rank"))
print(dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "regret")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", caption="Regret"))
print(dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "ratio")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", na_rep="-", caption="Ratio"))

\begin{table}
\centering
\caption{Rank}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1 &     3 &     5 &    15 &    25 \\
\midrule
 1 & 10.63 & 10.69 & 10.69 & 10.64 & 10.57 \\
 3 & 10.60 & 10.66 & 10.65 & 10.67 & 10.62 \\
 5 & 10.63 & 10.69 & 10.69 & 10.68 & 10.64 \\
15 & 10.78 & 10.84 & 10.85 & 10.79 & 10.74 \\
25 & 10.71 & 10.78 & 10.78 & 10.75 & 10.71 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{Regret}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &      1 &      3 &      5 &     15 &     25 \\
\midrule
 1 & 149.08 & 159.26 & 176.63 & 162.89 & 154.49 \\
 3 & 156.94 & 161.91 & 165.09 & 157.74 & 152.64 \\
 5 & 154.88 & 162.45 & 165.45 & 156.65 & 152.52 \\
15 & 151.33 & 163.55 & 171.64 & 159.74 & 154.53 \\
25 & 143.17 & 154.59 & 159.00 & 150.39 & 147.27 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{Ratio}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1

/tmp/ipykernel_916220/3484650822.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  print(dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", caption="Rank"))
/tmp/ipykernel_916220/3484650822.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", caption="Rank"))
/tmp/ipykernel_916220/3484650822.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index wi

In [61]:
print(dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", caption="Rank"))
print(dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "regret")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", caption="Regret"))
print(dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "ratio")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", na_rep="-", caption="Ratio"))

\begin{table}
\centering
\caption{Rank}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1 &     3 &     5 &    15 &   25 \\
\midrule
 1 & 26.10 & 19.14 & 16.21 & 10.96 & 7.13 \\
 3 & 10.99 &  7.43 &  6.40 &  3.60 & 2.43 \\
 5 &  6.87 &  4.44 &  3.66 &  1.80 & 1.24 \\
15 &  2.27 &  1.17 &  0.89 &  0.39 & 0.25 \\
25 &  1.23 &  0.61 &  0.43 &  0.16 & 0.10 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{Regret}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1 &     3 &     5 &    15 &    25 \\
\midrule
 1 & 43.95 & 34.37 & 29.75 & 21.24 & 15.26 \\
 3 & 22.11 & 16.28 & 14.60 &  8.69 &  6.65 \\
 5 & 15.78 & 11.25 &  9.92 &  5.29 &  4.13 \\
15 &  6.32 &  3.66 &  3.17 &  1.66 &  1.03 \\
25 &  3.90 &  2.00 &  1.55 &  0.66 &  0.39 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{Ratio}
\begin{tabular}{rrrrrr}
\toprule
 k & \multicolumn{5}{l}{r} \\
   &     1 &     3 &     5 &    15 &    25 \\


/tmp/ipykernel_916220/4145729495.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  print(dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", caption="Rank"))
/tmp/ipykernel_916220/4145729495.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).to_latex(index=False,float_format="%.2f", caption="Rank"))
/tmp/ipykernel_916220/4145729495.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index wi

In [75]:
m = pd.concat((
    dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).set_index("k"),
    dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "ratio")].drop(columns=["mode", "split", "metric"]).set_index("k"),
dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "regret")].drop(columns=["mode", "split", "metric"]).set_index("k")
), axis=1, keys=["rank", "ratio", "regret"])
print(m.to_latex(index=True,float_format="%.2f", na_rep="-", caption="Configuration-Configuration"))

\begin{table}
\centering
\caption{Configuration-Configuration}
\begin{tabular}{lrrrrrrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{rank} & \multicolumn{5}{l}{ratio} & \multicolumn{5}{l}{regret} \\
{} & \multicolumn{5}{l}{r} & \multicolumn{5}{l}{r} & \multicolumn{5}{l}{r} \\
{} &     1 &     3 &     5 &    15 &    25 &     1 &     3 &     5 &    15 &    25 &      1 &      3 &      5 &     15 &     25 \\
k  &       &       &       &       &       &       &       &       &       &       &        &        &        &        &        \\
\midrule
1  & 10.63 & 10.69 & 10.69 & 10.64 & 10.57 &     - &     - &     - &     - &     - & 149.08 & 159.26 & 176.63 & 162.89 & 154.49 \\
3  & 10.60 & 10.66 & 10.65 & 10.67 & 10.62 & 17.42 & 24.84 & 26.12 & 33.02 & 35.95 & 156.94 & 161.91 & 165.09 & 157.74 & 152.64 \\
5  & 10.63 & 10.69 & 10.69 & 10.68 & 10.64 & 17.67 & 25.03 & 26.65 & 34.16 & 38.08 & 154.88 & 162.45 & 165.45 & 156.65 & 152.52 \\
15 & 10.78 & 10.84 & 10.85 & 10.79 & 10.74 & 24.84 & 33.47 & 3

/tmp/ipykernel_916220/223827428.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).set_index("k"),
/tmp/ipykernel_916220/223827428.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "ratio")].drop(columns=["mode", "split", "metric"]).set_index("k"),
/tmp/ipykernel_916220/223827428.py:4: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dfmean[(dfmean["mode"] == "cc") & (dfmean["split"] == "test") & (dfmean["metric"] == "regret")].drop(columns=["mode", "split", "metric"]).set_index("k")
/tmp/ipykernel_916220/223827428.py:6: FutureWarning: In future versions `DataFrame.to_latex`

In [74]:
m = pd.concat((
    dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).set_index("k"),
    dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "ratio")].drop(columns=["mode", "split", "metric"]).set_index("k"),
dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "regret")].drop(columns=["mode", "split", "metric"]).set_index("k")
), axis=1, keys=["rank", "ratio", "regret"])
print(m.to_latex(index=True,float_format="%.2f", na_rep="-", caption="Input-Input"))

\begin{tabular}{lrrrrrrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{rank} & \multicolumn{5}{l}{ratio} & \multicolumn{5}{l}{regret} \\
{} & \multicolumn{5}{l}{r} & \multicolumn{5}{l}{r} & \multicolumn{5}{l}{r} \\
{} &     1 &     3 &     5 &    15 &   25 &     1 &     3 &     5 &    15 &    25 &      1 &     3 &     5 &    15 &    25 \\
k  &       &       &       &       &      &       &       &       &       &       &        &       &       &       &       \\
\midrule
1  & 26.10 & 19.14 & 16.21 & 10.96 & 7.13 &     - &     - &     - &     - &     - &  43.95 & 34.37 & 29.75 & 21.24 & 15.26 \\
3  & 10.99 &  7.43 &  6.40 &  3.60 & 2.43 & 23.78 & 29.85 & 33.15 & 44.23 & 53.18 &  22.11 & 16.28 & 14.60 &  8.69 &  6.65 \\
5  &  6.87 &  4.44 &  3.66 &  1.80 & 1.24 & 31.86 & 38.46 & 42.31 & 53.91 & 62.65 &  15.78 & 11.25 &  9.92 &  5.29 &  4.13 \\
15 &  2.27 &  1.17 &  0.89 &  0.39 & 0.25 & 50.90 & 57.82 & 61.98 & 73.35 & 80.17 &   6.32 &  3.66 &  3.17 &  1.66 &  1.03 \\
25 &  1.23 &  0.61 &  0.

/tmp/ipykernel_916220/2453670689.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "rank")].drop(columns=["mode", "split", "metric"]).set_index("k"),
/tmp/ipykernel_916220/2453670689.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "ratio")].drop(columns=["mode", "split", "metric"]).set_index("k"),
/tmp/ipykernel_916220/2453670689.py:4: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dfmean[(dfmean["mode"] == "ii") & (dfmean["split"] == "test") & (dfmean["metric"] == "regret")].drop(columns=["mode", "split", "metric"]).set_index("k")
/tmp/ipykernel_916220/2453670689.py:6: FutureWarning: In future versions `DataFrame.to_la